In [2]:
import pandas as pd
import re
from kiwipiepy import Kiwi # 💡 Mecab 대신 Kiwi를 import 합니다.
import os

# Kiwi 형태소 분석기 설치 (최초 1회만 실행)
try:
    import kiwipiepy
except ImportError:
    print("kiwipiepy 라이브러리를 설치합니다...")
    # 주피터 노트북 환경에서 pip install 실행
    import sys
    !{sys.executable} -m pip install kiwipiepy

# 1. 수집한 원본 데이터 불러오기
try:
    df = pd.read_csv('../data/raw_reviews.csv')
    print("----- 원본 데이터 -----")
    print(df.head())
except FileNotFoundError:
    print("오류: '../data/raw_reviews.csv' 파일을 찾을 수 없습니다. 01_crawling.ipynb를 먼저 실행했는지 확인하세요.")
    exit()

# 2. 텍스트 정제 함수 정의
def clean_text(text):
    if not isinstance(text, str):
        return ""
    cleaned = re.sub(r'[^가-힣A-Za-z0-9\s]', '', text)
    return cleaned

df['review_cleaned'] = df['review'].apply(clean_text)

# 3. Kiwi 형태소 분석기 초기화 및 분석/불용어 처리
kiwi = None
try:
    kiwi = Kiwi()
    print("\n\n🎉 Kiwi 형태소 분석기 초기화 성공! 🎉")

except Exception as e:
    print(f"\n\n❌ 오류: Kiwi 형태소 분석기 초기화에 실패했습니다: {e}")

# Kiwi가 성공적으로 초기화된 경우에만 다음 단계를 진행합니다.
if kiwi:
    stopwords = ['곳', '것', '저', '저희', '이', '그', '때', '수', '개', '분', '여기', '방문', '주문', '추천', '진짜', '너무', '완전', '정말']

    def tokenize_with_kiwi(text):
        # Kiwi를 이용해 텍스트를 토큰화하고, 일반 명사(NNG)와 고유 명사(NNP)만 추출합니다.
        result = kiwi.tokenize(text)
        tokens = [token.form for token in result if token.tag in ['NNG', 'NNP']]
        
        # 한 글자 단어와 불용어 리스트에 포함된 단어를 제거합니다.
        tokens = [word for word in tokens if len(word) > 1 and word not in stopwords]
        return tokens

    df['tokens'] = df['review_cleaned'].apply(tokenize_with_kiwi)

    # 4. 전처리된 데이터 저장
    df.to_csv('../data/processed_reviews.csv', index=False, encoding='utf-8-sig')

    print("\n\n----- 전처리 완료 후 데이터 -----")
    print(df[['review', 'tokens']].head())
    print("\n\n✅ 데이터 전처리가 완료되어 '../data/processed_reviews.csv' 파일로 저장했습니다.")



----- 원본 데이터 -----
  store_id store_name                                             review
0        A    잔잔 범계역점  언제나 애정하는 잔잔입니다:)\n오늘은\n다양하게 먹어봤어오! 모찌리도후는\n먹고싶...
1        A    잔잔 범계역점  인기많은 모찌리도후 오늘은 마지막으로 겟 했습니다:)\n해물야끼우동 처음 먹어보는데...
2        A    잔잔 범계역점  항상 인계동에서만 가봤다가 범계는 처음 가봤어요. 공휴일 저녁이라 역시 웨이팅이 있...
3        A    잔잔 범계역점  #범계술집#범계맛집#범계모임장소\n남편과 데이트 할때면 무조건 오는 잔잔~^^\n맛...
4        A    잔잔 범계역점  범계술집하면 믿고 먹는 잔잔이죠 !! 안주 뭘 시켜도 너무 맛있고 하이볼 음료 삼아...


🎉 Kiwi 형태소 분석기 초기화 성공! 🎉


----- 전처리 완료 후 데이터 -----
                                              review  \
0  언제나 애정하는 잔잔입니다:)\n오늘은\n다양하게 먹어봤어오! 모찌리도후는\n먹고싶...   
1  인기많은 모찌리도후 오늘은 마지막으로 겟 했습니다:)\n해물야끼우동 처음 먹어보는데...   
2  항상 인계동에서만 가봤다가 범계는 처음 가봤어요. 공휴일 저녁이라 역시 웨이팅이 있...   
3  #범계술집#범계맛집#범계모임장소\n남편과 데이트 할때면 무조건 오는 잔잔~^^\n맛...   
4  범계술집하면 믿고 먹는 잔잔이죠 !! 안주 뭘 시켜도 너무 맛있고 하이볼 음료 삼아...   

                                              tokens  
0  [애정, 오늘, 다양, 모찌리도후, 품절, 안쪽, 자리, 리모델링, 안주, 금액, ...  
1  [인기, 모찌리도후, 오늘, 마지막, 해물야끼, 우동, 처음, 범계, 